In [1]:
import glob
import os

import pandas as pd

from sqlalchemy import create_engine
import pymysql

import database_info

pymysql.install_as_MySQLdb()

In [2]:
engine, conn = database_info.getServerEngine()

In [3]:
glob.glob('../raw/south_korea/*.csv')

['../raw/south_korea\\20180101_20180331.csv',
 '../raw/south_korea\\20180401_20180630.csv',
 '../raw/south_korea\\20180701_20180930.csv',
 '../raw/south_korea\\20181001_20181231.csv',
 '../raw/south_korea\\20190101_20190331.csv',
 '../raw/south_korea\\20190401_20190630.csv',
 '../raw/south_korea\\20190701_20190930.csv',
 '../raw/south_korea\\20191001_20191231.csv',
 '../raw/south_korea\\20200101_20200331.csv']

In [4]:
pd.read_csv('../raw/south_korea\\20180101_20180331.csv', encoding='euc-kr').head(2)

,기준일시,공급능력(MW),현재수요(MW),최대예측수요(MW),공급예비력(MW),공급예비율(%),운영예비력(MW),운영예비율(%)
0,20180101000000,97257.0,60955.0,62000.0,36302.0,59.5553,26023.0,42.6921
1,20180101000500,97257.0,60964.3,62000.0,36292.7,59.5310,26013.7,42.6703


In [5]:
df = pd.DataFrame()
for file in sorted(glob.glob('../raw/south_korea/*.csv')):
    
    tmp = pd.read_csv(file, encoding='euc-kr')
    print(os.path.basename(file),':',len(tmp))
    df = pd.concat([df,tmp], axis=0, ignore_index=True)

20180101_20180331.csv : 25907
20180401_20180630.csv : 26166
20180701_20180930.csv : 26492
20181001_20181231.csv : 26493
20190101_20190331.csv : 25919
20190401_20190630.csv : 26205
20190701_20190930.csv : 26490
20191001_20191231.csv : 26491
20200101_20200331.csv : 26167


In [6]:
df

,기준일시,공급능력(MW),현재수요(MW),최대예측수요(MW),공급예비력(MW),공급예비율(%),운영예비력(MW),운영예비율(%)
0,20180101000000,97257.0,60955.0,62000.0,36302.0,59.5553,26023.00,42.6921
1,20180101000500,97257.0,60964.3,62000.0,36292.7,59.5310,26013.70,42.6703
2,20180101001000,97257.0,60745.9,62000.0,36511.1,60.1046,26232.10,43.1833
3,20180101001500,97257.0,60170.5,62000.0,37086.5,61.6357,26807.50,44.5526
4,20180101002000,97257.0,60067.7,62000.0,37189.3,61.9123,26910.30,44.7999
...,...,...,...,...,...,...,...,...
236325,20200331233500,86967.2,60207.4,64600.0,26760.4,44.4469,7863.41,13.0605
236326,20200331234000,86979.1,60055.0,64600.0,26960.2,44.8925,7612.62,12.6761
236327,20200331234500,86986.8,59598.0,64600.0,27430.9,46.0265,7975.05,13.3814
236328,20200331235000,86992.4,59368.0,64600.0,27683.8,46.6308,7812.69,13.1598


In [7]:
pwr = df[['기준일시','현재수요(MW)']]

In [8]:
pwr = pwr.rename(columns={'기준일시':'LoadDay','현재수요(MW)':'currPwr'})

In [9]:
pwr.LoadDay = pwr.LoadDay.astype(str)

In [10]:
pwr.LoadDay = pwr.LoadDay.map(lambda x:pd.Timestamp(int(x[:4]),int(x[4:6]),int(x[6:8]),int(x[8:10]),int(x[10:12])))

In [11]:
pwr

,LoadDay,currPwr
0,2018-01-01 00:00:00,60955.0
1,2018-01-01 00:05:00,60964.3
2,2018-01-01 00:10:00,60745.9
3,2018-01-01 00:15:00,60170.5
4,2018-01-01 00:20:00,60067.7
...,...,...
236325,2020-03-31 23:35:00,60207.4
236326,2020-03-31 23:40:00,60055.0
236327,2020-03-31 23:45:00,59598.0
236328,2020-03-31 23:50:00,59368.0


In [12]:
ref = pd.DataFrame({'LoadDay':pd.date_range('2018-01-01 00:00:00','2020-03-31 23:55:00', freq='5min')})

In [13]:
ref

,LoadDay
0,2018-01-01 00:00:00
1,2018-01-01 00:05:00
2,2018-01-01 00:10:00
3,2018-01-01 00:15:00
4,2018-01-01 00:20:00
...,...
236443,2020-03-31 23:35:00
236444,2020-03-31 23:40:00
236445,2020-03-31 23:45:00
236446,2020-03-31 23:50:00


In [14]:
ref = ref.join(pwr.set_index('LoadDay'), on='LoadDay')

In [15]:
ref

,LoadDay,currPwr
0,2018-01-01 00:00:00,60955.0
1,2018-01-01 00:05:00,60964.3
2,2018-01-01 00:10:00,60745.9
3,2018-01-01 00:15:00,60170.5
4,2018-01-01 00:20:00,60067.7
...,...,...
236443,2020-03-31 23:35:00,60207.4
236444,2020-03-31 23:40:00,60055.0
236445,2020-03-31 23:45:00,59598.0
236446,2020-03-31 23:50:00,59368.0


In [16]:
ref.isna().sum()

LoadDay      0
currPwr    118
dtype: int64

In [17]:
ref.to_sql(name='south_korea_kpx', con=conn)